In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn import linear_model

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import tree


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

### Load And Prepare Data

In [2]:
df_train = pd.read_csv('tmlcc-2021/train.csv')
df_test = pd.read_csv('tmlcc-2021/test.csv')
df_pre_train = pd.read_csv('tmlcc-2021/pretest.csv')

In [3]:
df_test.isna().sum(axis=0)

MOFname                                          0
volume [A^3]                                     0
weight [u]                                       0
surface_area [m^2/g]                             0
void_fraction                                    0
void_volume [cm^3/g]                             0
functional_groups                                0
metal_linker                                     0
organic_linker1                                  0
organic_linker2                                  0
topology                                         0
CO2/N2_selectivity                               0
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    0
dtype: int64

In [4]:
df_train.isna().sum(axis=0)

MOFname                                             0
volume [A^3]                                        0
weight [u]                                          0
surface_area [m^2/g]                                0
void_fraction                                       0
void_volume [cm^3/g]                                0
functional_groups                                 323
metal_linker                                        0
organic_linker1                                     0
organic_linker2                                     0
topology                                            0
CO2/N2_selectivity                                  0
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    2087
CO2_working_capacity [mL/g]                         0
dtype: int64

In [5]:
df_train['functional_groups'].isna().sum()

323

In [6]:
df = df_train.dropna()

In [9]:
df_train.replace([np.inf], np.nan, inplace=True)
df_train = df_train.dropna()

In [10]:
df_X = df_train.iloc[:, [1, 2, 3, 4, 5, 11, 12]]
df_y = df_train.iloc[:, 13]

df_test_X = df_test.iloc[:, [1, 2, 3, 4, 5, 11, 12]]

In [11]:
sd_scale = preprocessing.StandardScaler()
sd_scale.fit(df_X)

StandardScaler()

In [12]:
X = sd_scale.transform(df_X)
y = df_y

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [15]:
# Loading some example data
#X, y = X_train, y_train

# Training classifiers
reg1 = GradientBoostingRegressor()
reg2 = RandomForestRegressor(max_depth = 30, min_samples_split= 3, n_estimators= 1000, n_jobs= 5, random_state= 42)
reg3 = model = Pipeline([('poly', PolynomialFeatures(degree=4)),
                  ('linear', LinearRegression(fit_intercept=False))])
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
ereg = ereg.fit(X, y)

In [16]:
ereg.score(X, y)

0.9022107947666886

In [17]:
X_test = sd_scale.transform(df_test_X)
pred = ereg.predict(X_test)
ID = np.arange(68614, 85614)
pd.DataFrame({'id':ID, 'CO2_working_capacity [mL/g]': pred}).set_index('id').to_csv('submission.csv')

In [18]:
from zipfile import ZipFile

In [19]:
with ZipFile('model02.zip', mode='w') as zf:
        zf.write('submission.csv')